# OpenCALM SageMaker Inference with CTranslate2

[Open CALM](https://huggingface.co/cyberagent/open-calm-7b) を CTranslate2 で高速化し SageMaker でデプロイするサンプルコード。

検証は SageMake Studio Notebook で ml.m5.4xlarge 上で PyTorch 2.0.0 Python 3.10 CPU Optimized コンテナで行いました。

In [ ]:
!pip install "sagemaker>=2.143.0" -U
!pip install ctranslate2 transformers torch

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## Convert Model

モデルを CTranslate2 に最適化された形式に変換します。この処理はメモリを大きく利用するため十分なインスタンスサイズを選択してください。検証は m5.4xlarge で行いました。

In [ ]:
!rm -rf scripts/model
!ct2-transformers-converter --low_cpu_mem_usage --model cyberagent/open-calm-7b --quantization int8 --output_dir scripts/model

## Package and Upload Model

In [ ]:
!apt update -y
!apt install pigz -y

In [ ]:
%cd scripts
# !tar -czvf ../package.tar.gz *
!tar cv ./ | pigz -p 8 > ../package.tar.gz # 8 並列でアーカイブ
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"OpenCALM-Inference-CTranslate2")
model_path

## Deploy Model

In [ ]:
from sagemaker.serializers import JSONSerializer

endpoint_name = "OpenCALM-Inference-CTranslate"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="2.0",
    py_version='py310',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "tokenizer": "cyberagent/open-calm-7b",
            "model": "model",
            "prompt_input": "システム: {input}ユーザー: {instruction}<NL>システム: ",
            "prompt_no_input": "ユーザー: {instruction}<NL>システム: "
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
)

## Inference

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_client=Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)
data = {
    "instruction": """ヴァージン・オーストラリアはいつから運航を開始したのですか？完結に答えてください。""".replace("\n", "<NL>"),  # システム
    "input": """ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。""".replace("\n", "<NL>"),  # ユーザー
    "max_new_tokens": 64,
    "sampling_temperature": 0.3,
    "stop_ids": [0, 1],
}
response = predictor_client.predict(
    data=data
)
print(response.replace("<NL>", "\n"))

## Benchmark

1.36 s ± 320 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit response = predictor_client.predict(data=data)

In [ ]:
!pwd

In [ ]:
df = pd.read_json('/root/LLM/AutoModel/data/aio_02_dev_v1.0.jsonl', orient='records', lines=True)

In [ ]:
%%time
from tqdm import tqdm
import pandas as pd
import re

def inference(instruction, input):
    data = {
        "instruction": instruction,
        "input": input,
        "max_new_tokens": 8,
        "sampling_temperature": 0.1,
        # "do_sample": False,
        # "top_k": 500,
        # "top_p": 0.95,
        # "num_beams": 5,
        # "pad_token_id": 1,
        # "bos_token_id": 0,
        # "eos_token_is": 0,
        "stop_ids": [1, 0],
    }
    response = predictor_client.predict(
        data=data
    )
    return response

# Zero Shot
correct = 0
for idx, row in df.iterrows():
    prompt = row['question'] + "答えは「"
    result = inference("", prompt)
    # print(result)
    result = prompt + result
    try:
        result = re.findall("「(.*?)」", result)[-1]
    except IndexError:
        result = result
        print("longer output:", result)
    result = re.sub(r'[(].*[)]', "", result)
    if result in row['answers']:
        correct += 1
    else:
        print(result, row['answers'])
print(correct, "/", len(df))

## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()